<a href="https://colab.research.google.com/github/KatyaFonina/-6HTML/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%BA%D1%824_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pytest

In [ ]:
pip install pytest-sugar

In [ ]:
%%file petstore_api.py
import requests
import pprint
import time


class BaseRequest:
    def __init__(self, base_url):
        self.base_url = base_url
        # set headers, authorisation etc

    def _request(self, url, request_type, data=None, expected_error=False):
        stop_flag = False
        while not stop_flag:
            if request_type == 'GET':
                response = requests.get(url)
            elif request_type == 'POST':
                response = requests.post(url, json=data)
            elif request_type == 'PUT':
                response = requests.put(url, json=data)
            else:
                response = requests.delete(url)

            if not expected_error and response.status_code == 200:
                stop_flag = True
            elif expected_error:
                stop_flag = True

        # log part
        pprint.pprint(f'{request_type} example')
        pprint.pprint(response.url)
        pprint.pprint(response.status_code)
        pprint.pprint(response.reason)
        pprint.pprint(response.text)
        if response.text:  # Check if response has content
            try:
                pprint.pprint(response.json())
            except:
                pass
        pprint.pprint('**********')
        return response

    def get(self, endpoint, endpoint_id, expected_error=False):
        url = f'{self.base_url}/{endpoint}/{endpoint_id}'
        response = self._request(url, 'GET', expected_error=expected_error)
        return response.json()

    def post(self, endpoint, endpoint_id, body):
        url = f'{self.base_url}/{endpoint}/{endpoint_id}'
        response = self._request(url, 'POST', data=body)
        return response.json()

    def delete(self, endpoint, endpoint_id):
        url = f'{self.base_url}/{endpoint}/{endpoint_id}'
        response = self._request(url, 'DELETE')
        return response.json()


class UserAPI(BaseRequest):
    def __init__(self, base_url):
        super().__init__(base_url)

    def create_user(self, user_data):
        """Create user - POST /user"""
        url = f'{self.base_url}/user'
        response = self._request(url, 'POST', data=user_data)
        return response.json()

    def get_user_by_username(self, username, expected_error=False):
        """Get user by username - GET /user/{username}"""
        url = f'{self.base_url}/user/{username}'
        response = self._request(url, 'GET', expected_error=expected_error)
        return response.json()

    def update_user(self, username, user_data):
        """Update user - PUT /user/{username}"""
        url = f'{self.base_url}/user/{username}'
        response = self._request(url, 'PUT', data=user_data)
        return response.json()

    def delete_user(self, username):
        """Delete user - DELETE /user/{username}"""
        url = f'{self.base_url}/user/{username}'
        response = self._request(url, 'DELETE')
        return response.json()

    def login_user(self, username, password):
        """Logs user into the system - GET /user/login"""
        url = f'{self.base_url}/user/login?username={username}&password={password}'
        response = self._request(url, 'GET')
        return response.json()

    def logout_user(self):
        """Logs out current logged in user session - GET /user/logout"""
        url = f'{self.base_url}/user/logout'
        response = self._request(url, 'GET')
        return response.json()

    def create_with_list(self, users_list):
        """Creates list of users with given input array - POST /user/createWithList"""
        url = f'{self.base_url}/user/createWithList'
        response = self._request(url, 'POST', data=users_list)
        return response.json()


class StoreAPI(BaseRequest):
    def __init__(self, base_url):
        super().__init__(base_url)

    def get_inventory(self):
        """Returns pet inventories by status - GET /store/inventory"""
        url = f'{self.base_url}/store/inventory'
        response = self._request(url, 'GET')
        return response.json()

    def place_order(self, order_data):
        """Place an order for a pet - POST /store/order"""
        url = f'{self.base_url}/store/order'
        response = self._request(url, 'POST', data=order_data)
        return response.json()

    def get_order_by_id(self, order_id, expected_error=False):
        """Find purchase order by ID - GET /store/order/{orderId}"""
        url = f'{self.base_url}/store/order/{order_id}'
        response = self._request(url, 'GET', expected_error=expected_error)
        return response.json()

    def delete_order(self, order_id):
        """Delete purchase order by ID - DELETE /store/order/{orderId}"""
        url = f'{self.base_url}/store/order/{order_id}'
        response = self._request(url, 'DELETE')
        return response.json()

Overwriting petstore_api.py


In [ ]:
%%file test_petstore_api.py
import pytest
from petstore_api import UserAPI, StoreAPI
import time

BASE_URL_PETSTORE = 'https://petstore.swagger.io/v2'

class TestUserAPI:
    def setup_method(self):
        self.user_api = UserAPI(BASE_URL_PETSTORE)
        # Используем уникальные данные для каждого теста
        self.test_username = f"testuser_{int(time.time())}"
        self.user_data = {
            "id": int(time.time()),
            "username": self.test_username,
            "firstName": "Test",
            "lastName": "User",
            "email": f"{self.test_username}@example.com",
            "password": "password123",
            "phone": "1234567890",
            "userStatus": 1
        }

    def test_create_and_get_user(self):
        """Test creating a user and then retrieving it"""
        # Create user
        created_user = self.user_api.create_user(self.user_data)
        assert created_user['code'] == 200

        # Небольшая задержка для обновления данных на сервере
        time.sleep(1)

        # Get user
        user_info = self.user_api.get_user_by_username(self.test_username)
        assert user_info['username'] == self.test_username
        assert user_info['email'] == self.user_data['email']

    def test_update_user(self):
        """Test updating user information"""
        # First create user
        self.user_api.create_user(self.user_data)
        time.sleep(1)

        # Update user - используем те же данные но с обновленными полями
        updated_data = {
            "id": self.user_data["id"],
            "username": self.test_username,
            "firstName": "UpdatedName",
            "lastName": "UpdatedUser",
            "email": f"updated_{self.test_username}@example.com",
            "password": "newpassword123",
            "phone": "0987654321",
            "userStatus": 1
        }

        update_result = self.user_api.update_user(self.test_username, updated_data)
        assert update_result['code'] == 200

        # Verify update
        time.sleep(1)
        user_info = self.user_api.get_user_by_username(self.test_username)
        assert user_info['firstName'] == "UpdatedName"
        assert user_info['email'] == f"updated_{self.test_username}@example.com"

    def test_delete_user(self):
        """Test deleting a user"""
        # First create user
        self.user_api.create_user(self.user_data)
        time.sleep(1)

        # Delete user
        delete_result = self.user_api.delete_user(self.test_username)
        # Исправляем проверку - response.json() возвращает dict
        assert delete_result['message'] == self.test_username

        # Verify user is deleted
        user_info = self.user_api.get_user_by_username(self.test_username, expected_error=True)
        assert user_info['code'] == 1
        assert user_info['type'] == 'error'

    def test_user_login_logout(self):
        """Test user login and logout functionality"""
        # First create user
        self.user_api.create_user(self.user_data)
        time.sleep(1)

        # Login
        login_result = self.user_api.login_user(self.test_username, "password123")
        assert 'logged in' in login_result['message'].lower()

        # Logout
        logout_result = self.user_api.logout_user()
        assert logout_result['code'] == 200
        assert 'ok' in logout_result['message'].lower()


class TestStoreAPI:
    def setup_method(self):
        self.store_api = StoreAPI(BASE_URL_PETSTORE)
        # Используем уникальный ID для заказа
        self.order_id = int(time.time()) % 10000  # Ограничиваем размер числа
        self.order_data = {
            "id": self.order_id,
            "petId": 1,
            "quantity": 1,
            "shipDate": "2024-01-01T00:00:00.000Z",
            "status": "placed",
            "complete": True
        }

    def test_get_inventory(self):
        """Test getting store inventory"""
        inventory = self.store_api.get_inventory()
        assert isinstance(inventory, dict)
        # Проверяем что есть хотя бы один ключ (статус)
        assert len(inventory) > 0

    def test_place_and_get_order(self):
        """Test placing an order and then retrieving it"""
        # Place order
        placed_order = self.store_api.place_order(self.order_data)
        assert placed_order['id'] == self.order_data['id']
        assert placed_order['status'] == 'placed'

        # Get order
        order_info = self.store_api.get_order_by_id(self.order_data['id'])
        assert order_info['id'] == self.order_data['id']
        # Исправляем проверку - используем данные из ответа
        assert order_info['petId'] == placed_order['petId']

    def test_delete_order(self):
        """Test placing and then deleting an order"""
        # Place order first
        self.store_api.place_order(self.order_data)

        # Delete order
        delete_result = self.store_api.delete_order(self.order_data['id'])
        # Исправляем проверку - response.json() возвращает dict
        assert delete_result['message'] == str(self.order_data['id'])

        # Verify order is deleted
        order_info = self.store_api.get_order_by_id(self.order_data['id'], expected_error=True)
        assert order_info['code'] == 1
        assert order_info['type'] == 'error'

    def test_order_workflow(self):
        """Test complete order workflow"""
        # Place order
        order_result = self.store_api.place_order(self.order_data)
        assert order_result['status'] == 'placed'

        # Verify order exists
        order_info = self.store_api.get_order_by_id(self.order_data['id'])
        assert order_info['id'] == self.order_data['id']

        # Clean up - delete order
        delete_result = self.store_api.delete_order(self.order_data['id'])
        assert delete_result['message'] == str(self.order_data['id'])


# Run the tests
if __name__ == "__main__":
    pytest.main([__file__, "-v"])

Overwriting test_petstore_api.py


In [ ]:
!python -m pytest test_petstore_api.py

Test session starts (platform: linux, Python 3.12.12, pytest 8.4.2, pytest-sugar 1.1.1)
rootdir: /content
plugins: allure-pytest-2.15.0, sugar-1.1.1, typeguard-4.4.4, langsmith-0.4.38, anyio-4.11.0
collected 8 items                                                              

 test_petstore_api.py ✓✓✓✓✓✓✓✓                                   100% ██████████

Results (7.12s):
       8 passed
